# TP1: Le probleme d'apprentissage

**IFT6390 - Fondements de l'apprentissage machine**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pierrelux/mlbook/blob/main/exercises/tp1_learning_problem.ipynb)

Ce notebook accompagne le [Chapitre 2: Le probleme d'apprentissage](https://pierrelux.github.io/mlbook/learning-problem).

## Objectifs

À la fin de ce TP, vous serez en mesure de:
- Calculer l'erreur quadratique moyenne (MSE)
- Observer le phénomène de surapprentissage avec des polynômes
- Comprendre le compromis biais-variance
- Implémenter la régularisation Ridge

---

## Partie 0: Configuration

Exécutez cette cellule pour importer les bibliothèques nécessaires.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', message='Polyfit may be poorly conditioned')

# Pour de jolis graphiques
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['font.size'] = 12

print("✅ Configuration terminée!")

## Partie 1: Les données de freinage

Nous utilisons les données classiques d'Ezekiel (1930): la distance de freinage d'un véhicule en fonction de sa vitesse.

**Question préliminaire**: Quelle relation attendez-vous entre vitesse et distance de freinage? Linéaire? Quadratique? Autre?

In [ ]:
# Données de freinage (Ezekiel, 1930): vitesse (mph) vs distance d'arrêt (ft)
speed = np.array([4, 4, 7, 7, 8, 9, 10, 10, 10, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14,
                  14, 14, 14, 15, 15, 15, 16, 16, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19,
                  20, 20, 20, 20, 20, 22, 23, 24, 24, 24, 24, 25], dtype=float)
dist = np.array([2, 10, 4, 22, 16, 10, 18, 26, 34, 17, 28, 14, 20, 24, 28, 26, 34, 34, 46,
                 26, 36, 60, 80, 20, 26, 54, 32, 40, 32, 40, 50, 42, 56, 76, 84, 36, 46,
                 68, 32, 48, 52, 56, 64, 66, 54, 70, 92, 93, 120, 85], dtype=float)

print(f"Nombre d'observations: {len(speed)}")
print(f"Vitesse: min={speed.min():.0f}, max={speed.max():.0f} mph")
print(f"Distance: min={dist.min():.0f}, max={dist.max():.0f} ft")

In [ ]:
# Visualisation des données
plt.figure(figsize=(8, 5))
plt.scatter(speed, dist, alpha=0.7, s=50)
plt.xlabel('Vitesse (mph)')
plt.ylabel('Distance de freinage (ft)')
plt.title('Données de freinage')
plt.grid(True, alpha=0.3)
plt.show()

---
## Partie 2: Ajuster un polynôme

Nous allons ajuster des polynômes de différents degrés aux données.

### 2.1 Ajustement simple

In [ ]:
# Ajuster un polynôme de degré 2
degree = 2
coeffs = np.polyfit(speed, dist, degree)

print(f"Coefficients du polynôme de degré {degree}:")
for i, c in enumerate(coeffs):
    print(f"  coefficient de x^{degree-i}: {c:.4f}")

In [ ]:
# Visualiser l'ajustement
plt.figure(figsize=(8, 5))
plt.scatter(speed, dist, alpha=0.7, s=50, label='Données')

# Grille pour tracer la courbe
speed_grid = np.linspace(0, 30, 100)
dist_pred = np.polyval(coeffs, speed_grid)
plt.plot(speed_grid, dist_pred, 'r-', linewidth=2, label=f'Polynôme degré {degree}')

plt.xlabel('Vitesse (mph)')
plt.ylabel('Distance de freinage (ft)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 📝 Exercice 1: Calculer l'erreur quadratique moyenne (MSE)

L'erreur quadratique moyenne est définie comme:

$$\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

**Complétez la fonction ci-dessous:**

In [ ]:
def compute_mse(y_true, y_pred):
    """
    Calcule l'erreur quadratique moyenne.
    
    Args:
        y_true: valeurs réelles (array)
        y_pred: prédictions (array)
    
    Returns:
        MSE (float)
    """
    # ============================================
    # TODO: Complétez cette fonction
    # Indice: utilisez np.mean() et l'opérateur **2
    # ============================================
    
    mse = None  # <- Remplacez None par votre code
    
    return mse

In [ ]:
# Test de votre fonction
predictions = np.polyval(coeffs, speed)
mse = compute_mse(dist, predictions)

if mse is not None:
    print(f"MSE pour le polynôme de degré {degree}: {mse:.2f}")
    
    # Vérification
    expected_mse = np.mean((dist - predictions)**2)
    if np.isclose(mse, expected_mse):
        print("✅ Correct!")
    else:
        print(f"❌ Attendu: {expected_mse:.2f}")
else:
    print("⚠️ La fonction retourne None. Complétez le code!")

<details>
<summary>💡 Cliquez pour voir la solution</summary>

```python
def compute_mse(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    return mse
```
</details>

### 📝 Exercice 2: Explorer différents degrés

**Modifiez la variable `degree` ci-dessous et observez:**
- Comment change la courbe?
- Comment change le MSE?

In [ ]:
# ============================================
# TODO: Essayez différentes valeurs: 1, 2, 5, 10, 15, 20
# ============================================
degree = 2  # <- Modifiez cette valeur!

# Ajustement
coeffs = np.polyfit(speed, dist, degree)
predictions = np.polyval(coeffs, speed)
mse = np.mean((dist - predictions)**2)

# Visualisation
plt.figure(figsize=(10, 5))
plt.scatter(speed, dist, alpha=0.7, s=50, label='Données')

speed_grid = np.linspace(3, 26, 200)
pred_grid = np.polyval(coeffs, speed_grid)
pred_grid = np.clip(pred_grid, -50, 200)  # Limiter pour la visualisation

plt.plot(speed_grid, pred_grid, 'r-', linewidth=2, label=f'Polynôme degré {degree}')
plt.xlabel('Vitesse (mph)')
plt.ylabel('Distance de freinage (ft)')
plt.title(f'Degré {degree} — MSE = {mse:.1f}')
plt.legend()
plt.ylim(-20, 150)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nMSE: {mse:.2f}")

**❓ Questions de réflexion:**
1. Quel degré donne le MSE le plus bas?
2. Le polynôme de degré 20 semble-t-il raisonnable pour prédire à de nouvelles vitesses?
3. C'est quoi le problème avec minimiser uniquement le MSE sur les données d'entraînement?

---
## Partie 3: Train/Test Split — Le surapprentissage

Pour détecter le **surapprentissage**, nous séparons les données en:
- **Ensemble d'entraînement** (70%): pour ajuster le modèle
- **Ensemble de test** (30%): pour évaluer la généralisation

In [ ]:
# Séparation train/test
np.random.seed(42)  # Pour reproductibilité
indices = np.random.permutation(len(speed))
n_train = 35

train_idx = indices[:n_train]
test_idx = indices[n_train:]

speed_train, dist_train = speed[train_idx], dist[train_idx]
speed_test, dist_test = speed[test_idx], dist[test_idx]

print(f"Entraînement: {len(speed_train)} exemples")
print(f"Test: {len(speed_test)} exemples")

In [ ]:
# Visualisation de la séparation
plt.figure(figsize=(8, 5))
plt.scatter(speed_train, dist_train, alpha=0.7, s=50, label='Entraînement')
plt.scatter(speed_test, dist_test, alpha=0.7, s=50, marker='s', label='Test')
plt.xlabel('Vitesse (mph)')
plt.ylabel('Distance de freinage (ft)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 📝 Exercice 3: Courbe biais-variance

**Complétez le code pour calculer l'erreur sur train ET test pour chaque degré:**

In [ ]:
degrees = range(1, 16)
train_errors = []
test_errors = []

for deg in degrees:
    # Ajuster sur l'ensemble d'entraînement
    coeffs = np.polyfit(speed_train, dist_train, deg)
    
    # ============================================
    # TODO: Calculez les prédictions et erreurs
    # ============================================
    
    # Prédictions sur train
    pred_train = None  # <- Complétez avec np.polyval(...)
    
    # Prédictions sur test
    pred_test = None  # <- Complétez avec np.polyval(...)
    
    # MSE sur train
    mse_train = None  # <- Complétez
    
    # MSE sur test
    mse_test = None  # <- Complétez
    
    train_errors.append(mse_train)
    test_errors.append(mse_test)

In [ ]:
# Visualisation (exécutez après avoir complété le code ci-dessus)
if None not in train_errors and None not in test_errors:
    plt.figure(figsize=(10, 6))
    plt.plot(degrees, train_errors, 'o-', linewidth=2, markersize=8, label='Erreur entraînement')
    plt.plot(degrees, test_errors, 's-', linewidth=2, markersize=8, label='Erreur test')
    plt.yscale('log')
    plt.xlabel('Degré du polynôme (complexité)')
    plt.ylabel('MSE (échelle log)')
    plt.title('Compromis biais-variance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, 16, 2))
    plt.show()
    
    # Trouver le meilleur degré
    best_deg = degrees[np.argmin(test_errors)]
    print(f"\n🎯 Meilleur degré (selon erreur test): {best_deg}")
    print(f"   MSE train: {train_errors[best_deg-1]:.1f}")
    print(f"   MSE test: {test_errors[best_deg-1]:.1f}")
else:
    print("⚠️ Complétez le code de l'exercice 3!")

<details>
<summary>💡 Cliquez pour voir la solution</summary>

```python
# Prédictions sur train
pred_train = np.polyval(coeffs, speed_train)

# Prédictions sur test
pred_test = np.polyval(coeffs, speed_test)

# MSE sur train
mse_train = np.mean((dist_train - pred_train)**2)

# MSE sur test
mse_test = np.mean((dist_test - pred_test)**2)
```
</details>

---
## Partie 4: Régularisation Ridge

Au lieu de choisir un degré bas, on peut utiliser un polynôme de **haut degré** avec **régularisation**.

La régularisation Ridge ajoute une pénalité sur les coefficients:

$$\hat{\boldsymbol{\theta}}_{\text{Ridge}} = \arg\min_{\boldsymbol{\theta}} \left[ \sum_{i=1}^{N} (y_i - \boldsymbol{\theta}^\top \mathbf{x}_i)^2 + \lambda \|\boldsymbol{\theta}\|^2 \right]$$

La solution en forme fermée est:

$$\hat{\boldsymbol{\theta}}_{\text{Ridge}} = (\mathbf{X}^\top \mathbf{X} + \lambda \mathbf{I})^{-1} \mathbf{X}^\top \mathbf{y}$$

### 📝 Exercice 4: Implémenter Ridge Regression

**Complétez la fonction ci-dessous:**

In [ ]:
def polynomial_features(x, degree):
    """
    Crée la matrice de caractéristiques polynomiales.
    
    Args:
        x: vecteur d'entrées (n,)
        degree: degré du polynôme
    
    Returns:
        X: matrice (n, degree+1) avec colonnes [1, x, x², ..., x^degree]
    """
    n = len(x)
    X = np.zeros((n, degree + 1))
    for d in range(degree + 1):
        X[:, d] = x ** d
    return X


def ridge_regression(X, y, lambda_reg):
    """
    Calcule les coefficients Ridge.
    
    Args:
        X: matrice de features (n, d)
        y: vecteur cible (n,)
        lambda_reg: coefficient de régularisation
    
    Returns:
        theta: coefficients (d,)
    """
    # ============================================
    # TODO: Implémentez la formule Ridge
    # theta = (X'X + lambda*I)^(-1) X'y
    # 
    # Indices:
    # - X.T pour la transposée
    # - @ pour le produit matriciel
    # - np.eye(n) pour la matrice identité n×n
    # - np.linalg.inv() pour l'inverse
    # ============================================
    
    d = X.shape[1]  # nombre de features
    
    theta = None  # <- Remplacez par votre code
    
    return theta

In [ ]:
# Test de votre implémentation
degree = 10
lambda_reg = 1.0

X_train = polynomial_features(speed_train, degree)
theta = ridge_regression(X_train, dist_train, lambda_reg)

if theta is not None:
    print(f"Coefficients Ridge (degré={degree}, λ={lambda_reg}):")
    print(f"  Norme des coefficients: {np.linalg.norm(theta):.2f}")
    
    # Prédictions
    X_test = polynomial_features(speed_test, degree)
    pred_test = X_test @ theta
    mse = np.mean((dist_test - pred_test)**2)
    print(f"  MSE test: {mse:.1f}")
    print("\n✅ Implémentation correcte!")
else:
    print("⚠️ Complétez la fonction ridge_regression!")

<details>
<summary>💡 Cliquez pour voir la solution</summary>

```python
def ridge_regression(X, y, lambda_reg):
    d = X.shape[1]
    I = np.eye(d)
    theta = np.linalg.inv(X.T @ X + lambda_reg * I) @ X.T @ y
    return theta
```
</details>

### 📝 Exercice 5: Trouver le meilleur λ

**Explorez différentes valeurs de λ pour un polynôme de degré 10:**

In [ ]:
degree = 10
X_train = polynomial_features(speed_train, degree)
X_test = polynomial_features(speed_test, degree)

lambdas = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
results = []

print(f"{'λ':>10} | {'MSE Train':>10} | {'MSE Test':>10} | {'||θ||':>10}")
print("-" * 50)

for lam in lambdas:
    theta = ridge_regression(X_train, dist_train, lam)
    if theta is not None:
        mse_train = np.mean((dist_train - X_train @ theta)**2)
        mse_test = np.mean((dist_test - X_test @ theta)**2)
        norm_theta = np.linalg.norm(theta)
        results.append((lam, mse_train, mse_test, norm_theta))
        print(f"{lam:>10.3f} | {mse_train:>10.1f} | {mse_test:>10.1f} | {norm_theta:>10.1f}")

In [ ]:
# Visualisation de l'effet de λ
if len(results) > 0:
    lambdas_plot = [r[0] for r in results]
    train_plot = [r[1] for r in results]
    test_plot = [r[2] for r in results]
    
    plt.figure(figsize=(10, 5))
    plt.semilogx([max(l, 1e-4) for l in lambdas_plot], train_plot, 'o-', label='Train')
    plt.semilogx([max(l, 1e-4) for l in lambdas_plot], test_plot, 's-', label='Test')
    plt.xlabel('λ (échelle log)')
    plt.ylabel('MSE')
    plt.title(f'Effet de la régularisation Ridge (degré={degree})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

**❓ Questions:**
1. Quelle valeur de λ minimise l'erreur test?
2. Que se passe-t-il quand λ → 0? Et quand λ → ∞?
3. Comment la norme des coefficients ||θ|| change avec λ?

---
## 🎯 Récapitulatif

Dans ce TP, vous avez appris:

1. **MSE**: L'erreur quadratique moyenne mesure la qualité des prédictions

2. **Surapprentissage**: Un modèle trop complexe mémorise le bruit
   - Erreur train ↓ mais erreur test ↑
   
3. **Compromis biais-variance**: 
   - Modèle simple = biais élevé (sous-apprentissage)
   - Modèle complexe = variance élevée (surapprentissage)
   
4. **Régularisation Ridge**: Pénalise les grands coefficients
   - Permet d'utiliser des modèles complexes sans surapprentissage
   - λ contrôle la force de la régularisation

---

**📚 Pour aller plus loin**: [Chapitre complet sur le site du cours](https://pierrelux.github.io/mlbook/learning-problem)